In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:

from tensorflow.keras.applications import InceptionV3
Inception_model=InceptionV3(include_top=False,weights='imagenet',input_shape=(224,224,3),classes=2,classifier_activation='sigmoid')

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
Inception_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_7[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train=ImageDataGenerator(rescale=1./255,
                            rotation_range=5,
                            zoom_range=.2,
                            horizontal_flip=True,validation_split=0.2)
test=ImageDataGenerator(rescale=1./255)
train_data=train.flow_from_directory('/content/gdrive/MyDrive/dataset/datasetweek2/DogVsCat/training_set',color_mode='rgb',
                                        target_size=(224,224),
                                        batch_size=1,
                                        class_mode='binary',
                                        shuffle=True,subset="training")
val_data=train.flow_from_directory('/content/gdrive/MyDrive/dataset/datasetweek2/DogVsCat/training_set',color_mode='rgb',
                                        target_size=(224,224),
                                        batch_size=1,
                                        class_mode='binary',
                                        shuffle=True,subset="validation")
test_data=test.flow_from_directory('/content/gdrive/MyDrive/dataset/datasetweek2/DogVsCat/test_set',color_mode='rgb',
                                        target_size=(224,224),
                                        batch_size=1,
                                        class_mode='binary',
                                        shuffle=True)

Found 6404 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [ ]:
for layer in Inception_model.layers:
    layer.trainable = False

In [ ]:
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
x = Flatten()(Inception_model.output)
x=Dense(4096,activation='relu')(x)

In [ ]:

prediction=Dense(1, activation='sigmoid')(x)


In [ ]:
model = Model(inputs=Inception_model.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_7[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                           

In [ ]:
model.compile(
  loss='binary_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)
#Early stopping to avoid overfitting of model

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("inception_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [ ]:
# fit the model
history = model.fit(
 train_data,
  
  validation_data=val_data,
  epochs=10,steps_per_epoch=len(train_data)//32,
  callbacks=[early_stop,checkpoint],
  batch_size=32,shuffle=True)

Epoch 1/10
200/200 [==============================] - ETA: 0s - loss: 35.6219 - accuracy: 0.8900

200/200 [==============================] - 69s 320ms/step - loss: 35.6219 - accuracy: 0.8900 - val_loss: 3.6202 - val_accuracy: 0.9706
Epoch 2/10
200/200 [==============================] - ETA: 0s - loss: 8.8139 - accuracy: 0.9400

200/200 [==============================] - 52s 262ms/step - loss: 8.8139 - accuracy: 0.9400 - val_loss: 0.9973 - val_accuracy: 0.9688
Epoch 3/10
200/200 [==============================] - ETA: 0s - loss: 1.2900 - accuracy: 0.9650

200/200 [==============================] - 56s 280ms/step - loss: 1.2900 - accuracy: 0.9650 - val_loss: 1.0728 - val_accuracy: 0.9538
Epoch 4/10
200/200 [==============================] - ETA: 0s - loss: 1.6364 - accuracy: 0.9150

200/200 [==============================] - 58s 290ms/step - loss: 1.6364 - accuracy: 0.9150 - val_loss: 0.2035 - val_accuracy: 0.9738
Epoch 5/10
200/200 [==============================] - ETA: 0s - loss: 0.5096 - accuracy: 0.9350

...

200/200 [==============================] - 56s 279ms/step - loss: 0.2349 - accuracy: 0.9700 - val_loss: 0.8147 - val_accuracy: 0.9781
Epoch 7/10
200/200 [==============================] - ETA: 0s - loss: 1.3359 - accuracy: 0.9400

200/200 [==============================] - 58s 291ms/step - loss: 1.3359 - accuracy: 0.9400 - val_loss: 0.0850 - val_accuracy: 0.9763
Epoch 8/10
200/200 [==============================] - ETA: 0s - loss: 0.0444 - accuracy: 0.9900

200/200 [==============================] - 54s 272ms/step - loss: 0.0444 - accuracy: 0.9900 - val_loss: 0.1108 - val_accuracy: 0.9738
Epoch 9/10
200/200 [==============================] - ETA: 0s - loss: 0.0482 - accuracy: 0.9900

200/200 [==============================] - 54s 270ms/step - loss: 0.0482 - accuracy: 0.9900 - val_loss: 0.1600 - val_accuracy: 0.9719
Epoch 10/10
200/200 [==============================] - ETA: 0s - loss: 0.1589 - accuracy: 0.9750

200/200 [==============================] - 57s 287ms/step - loss: 0.1589 - accuracy: 0.9750 - val_loss: 13.9658 - val_accuracy: 0.5603


In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import numpy as np
#predict
y_pred=model.predict(test_data)
y_pred=np.argmax(y_pred,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_data.classes,y_pred)

array([[1011,    0],
       [1012,    0]])

In [ ]:

model.save("/content/drive/MyDrive/inception_dogsVscats.h5")
print("Saved model to disk")
model.save_weights("/content/drive/MyDrive/inception_dogsVscats.h5")

Saved model to disk
